In [1]:
!pip install --pre torch --index-url https://download.pytorch.org/whl/nightly/cpu -q
!pip install git+https://github.com/huggingface/transformers.git -q
!pip install git+https://github.com/deepmind/optax.git -q
!pip install jax jax-smi einops flax optax fire mypy wandb matplotlib tqdm types-tqdm pdoc3 tf-nightly accelerate sentencepiece protobuf~=3.20.0 -q
!pip install -U "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html -q


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.7.0 requires keras-core, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.4 which is incompatible.
tensorflow 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.3.2 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!rm -r /kaggle/working/falcon-jax
!git clone https://github.com/defdet/falcon-jax 
!cp -R /kaggle/working/falcon-jax/lib .

rm: cannot remove '/kaggle/working/falcon-jax': No such file or directory
Cloning into 'falcon-jax'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 275 (delta 119), reused 0 (delta 0), pack-reused 85
Receiving objects: 100% (275/275), 91.64 KiB | 3.67 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [3]:
import jax
import jax.numpy as jnp
import jax.random as rand
from transformers import AutoTokenizer
from tqdm import tqdm
from lib.falcon import Falcon, FalconModel, Attention, Decoder, LayerNorm, DecoderBlock
from lib.logits_processing import PresencePenaltyProcessor, TopKSampler, TopPSampler, make_logits_processor
from lib.param_utils import load_params
from lib.multihost_utils import shard_model_params
from lib.seeding import BEST_INTEGER
from lib.generation import generate

# Converting LLAMA to JAX consumes significant amount of time and resources so it's best to convert it beforehand
def load_params_from_disk() -> Falcon:
    cpu_device = jax.devices('cpu')[0]
    with jax.default_device(cpu_device):
        params = load_params('/kaggle/input/pure-falcon-7b-jax/Falcon-7B-JAX.pickle')
        params = jax.tree_map(lambda x: x.astype(jnp.bfloat16), params)
    params = shard_model_params(params)
    return params

top_k = 6
params = load_params_from_disk()
print('Successfully loaded model parameters!')

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0205 05:49:13.298739468     421 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {created_time:"2024-02-05T05:49:13.298722817+00:00", grpc_status:2}


Successfully loaded model parameters!


In [4]:
key = rand.key(BEST_INTEGER, impl='rbg')
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b', padding_side='left')
tokenizer.pad_token = tokenizer.eos_token
logits_processor = make_logits_processor(
    PresencePenaltyProcessor(penalty=0.05),
    TopKSampler(top_k=top_k),
        # TopPSampler(top_p=top_p),
)
batch = ['Tax evasion should not be a ', 'Tax evasion is not a '] # It's a pre-trained model, generation is a natural continiuation of prompt

tokenizer_config.json: 100%|██████████| 287/287 [00:00<00:00, 1.05MB/s]
tokenizer.json: 100%|██████████| 2.73M/2.73M [00:00<00:00, 38.9MB/s]
special_tokens_map.json: 100%|██████████| 281/281 [00:00<00:00, 1.51MB/s]


In [6]:
import einops as op
generated = []
for _ in tqdm(range(10)):
    key, subkey = rand.split(key)
    generated_batch = generate(batch, tokenizer, params, logits_processor, max_len=64, key=subkey)
    generated.append(generated_batch)
print(generated[0])

100%|██████████| 10/10 [00:15<00:00,  1.59s/it]

[' evasion should not be a "victimless crime", a top judge has warned.\nLord Justice Leggatt said the courts must take a "zero tolerance" approach to those who try to hide income or assets from HM Revenue and Customs (HMRC).\nSpeaking at a seminar in London, he', 'Tax evasion is not a “victimless crime.” It harms the economy and the government services and benefits that we need to thrive.\nThe IRS and states’ revenue collection agencies collect taxes to fund essential public services, such as education and health care. But when individuals and businesses cheat on their taxes, they dep']
